# Introduction to Modeling Libraries 

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## Interfacing Between pandas and Model Code

In [2]:
import pandas as pd
import numpy as np
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
print(data)
print(data.columns)
data.values

   x0    x1    y
0   1  0.01 -1.5
1   2 -0.01  0.0
2   3  0.25  3.6
3   4 -4.10  1.3
4   5  0.00 -2.0
Index(['x0', 'x1', 'y'], dtype='object')


array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [3]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [4]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [5]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [6]:
dummies = pd.get_dummies(data.category, prefix='category')
print(dummies)
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

   category_a  category_b
0           1           0
1           0           1
2           1           0
3           1           0
4           0           1


,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Creating Model Descriptions with Patsy

y ~ x0 + x1

In [7]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
print(data)
import patsy
y, X = patsy.dmatrices('y ~ x0 + x1', data)

   x0    x1    y
0   1  0.01 -1.5
1   2 -0.01  0.0
2   3  0.25  3.6
3   4 -4.10  1.3
4   5  0.00 -2.0


In [8]:
print(y)
X

[[-1.5]
 [ 0. ]
 [ 3.6]
 [ 1.3]
 [-2. ]]


DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [9]:
print(np.asarray(y))
np.asarray(X)

[[-1.5]
 [ 0. ]
 [ 3.6]
 [ 1.3]
 [-2. ]]


array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [10]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [11]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

In [12]:
print(coef)
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

[[ 0.3129]
 [-0.0791]
 [-0.2655]]


Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas

In [13]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [14]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [15]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [16]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical Data and Patsy

In [17]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
print(data)
y, X = patsy.dmatrices('v2 ~ key1', data)
X

  key1  key2  v1   v2
0    a     0   1 -1.0
1    a     1   2  0.0
2    b     0   3  2.5
3    b     1   4 -0.5
4    a     0   5  4.0
5    b     1   6 -1.2
6    a     0   7  0.2
7    b     0   8 -1.7


DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [18]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [19]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [20]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
print(data)
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
print(X)
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

  key1  key2  v1   v2
0    a  zero   1 -1.0
1    a   one   2  0.0
2    b  zero   3  2.5
3    b   one   4 -0.5
4    a  zero   5  4.0
5    b   one   6 -1.2
6    a  zero   7  0.2
7    b  zero   8 -1.7
[[ 1.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  1.  0.]
 [ 1.  0.  1.]
 [ 1.  1.  0.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]]


DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## Introduction to statsmodels

### Estimating Linear Models

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\ndwad\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [22]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# For reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [24]:
print(X[:5])
y[:5]

[[-0.1295 -1.2128  0.5042]
 [ 0.3029 -0.4357 -0.2542]
 [-0.3285 -0.0253  0.1384]
 [-0.3515 -0.7196 -0.2582]
 [ 1.2433 -0.3738 -0.5226]]


array([ 0.4279, -0.6735, -0.0909, -0.4895, -0.1289])

In [25]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.    , -0.1295, -1.2128,  0.5042],
       [ 1.    ,  0.3029, -0.4357, -0.2542],
       [ 1.    , -0.3285, -0.0253,  0.1384],
       [ 1.    , -0.3515, -0.7196, -0.2582],
       [ 1.    ,  1.2433, -0.3738, -0.5226]])

In [38]:
model = sm.OLS(y, X)

In [39]:
results = model.fit()
results.params

array([ 0.1783,  0.223 ,  0.501 ])

In [40]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Mon, 18 Dec 2017   Prob (F-statistic):           7.44e-12
Time:                        18:28:12   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

In [41]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [42]:
results = smf.ols('y ~ col0 + col1 + col2 + 0', data=data).fit()
print(results.params)
results.tvalues

col0    0.178261
col1    0.223040
col2    0.500951
dtype: float64


col0    3.364123
col1    4.818454
col2    6.237074
dtype: float64

In [43]:
results.predict(data[:5])

0   -0.040979
1   -0.170522
2    0.005101
3   -0.352508
4   -0.123557
dtype: float64

### Estimating Time Series Processes

In [58]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    print(values[-1], values[-2])
    values.append(new_x)

4 4
1.86977018119 4
0.0197113590032 1.86977018119
-1.67159535259 0.0197113590032
-1.12411087047 -1.67159535259
-0.142162188313 -1.12411087047
0.297349228149 -0.142162188313
0.370273631113 0.297349228149
0.42786720966 0.370273631113
0.158098962757 0.42786720966
-0.0151432430633 0.158098962757
0.0364606394735 -0.0151432430633
-0.0243883154834 0.0364606394735
-0.137803043419 -0.0243883154834
-0.0939193896473 -0.137803043419
0.0348525196959 -0.0939193896473
0.186951288369 0.0348525196959
-0.00324366209781 0.186951288369
-0.332600535535 -0.00324366209781
-0.379260610586 -0.332600535535
0.207708891914 -0.379260610586
-0.0183968427035 0.207708891914
0.589992716694 -0.0183968427035
0.901300111648 0.589992716694
0.982025009256 0.901300111648
0.521200916008 0.982025009256
-0.0918941919441 0.521200916008
-0.0139525672011 -0.0918941919441
-0.0153852623102 -0.0139525672011
0.142654373698 -0.0153852623102
0.106073590382 0.142654373698
0.267350367427 0.106073590382
-0.0685270950608 0.267350367427
-0.

-0.0751022639727 0.698729727944
0.0172641058786 -0.0751022639727
0.0540240588884 0.0172641058786
-0.117304465498 0.0540240588884
0.258189902452 -0.117304465498
-0.478290649142 0.258189902452
-0.759709291758 -0.478290649142
-0.561261787065 -0.759709291758
-0.0691170855322 -0.561261787065
-0.412536300051 -0.0691170855322
-0.119355588983 -0.412536300051
-0.362157052799 -0.119355588983
-0.204933820495 -0.362157052799
-0.0628008957807 -0.204933820495
-0.363936134534 -0.0628008957807
-0.610323811638 -0.363936134534
-1.00473382322 -0.610323811638
-0.594923410955 -1.00473382322
-0.224452260031 -0.594923410955
0.246691147619 -0.224452260031
0.608919380002 0.246691147619
0.497649825409 0.608919380002
0.656876928178 0.497649825409
-0.114587295517 0.656876928178
-0.443541445798 -0.114587295517
-0.274424886093 -0.443541445798
-0.182571082006 -0.274424886093
0.41990667918 -0.182571082006
0.517201774236 0.41990667918
0.340564212414 0.517201774236
-0.108551030445 0.340564212414
0.349274523892 -0.10855

In [53]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

In [54]:
results.params

array([-0.0062,  0.7845, -0.4085, -0.0136,  0.015 ,  0.0143])

## Introduction to scikit-learn

In [59]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [60]:
print(train.isnull().sum())
test.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [61]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [62]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [63]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
print(X_train[:5])
y_train[:5]

[[  3.   0.  22.]
 [  1.   1.  38.]
 [  3.   1.  26.]
 [  1.   1.  35.]
 [  3.   0.  35.]]


array([0, 1, 1, 1, 0], dtype=int64)

In [64]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [65]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [66]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

(y_true == y_predict).mean()

In [67]:
from sklearn.linear_model import LogisticRegressionCV
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [68]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([ 0.7723,  0.8027,  0.7703,  0.7883])

## Continuing Your Education

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS